# install simple salesforce

In [ ]:
#pip install simple_salesforce

In [1]:
#Prerequisites

import sys
!{sys.executable} -m pip install --user networkx
!{sys.executable} -m pip install --user numpy
!{sys.executable} -m pip install --user pandas

import networkx as nx
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import requests
import csv
import glob
import scipy.sparse

from simple_salesforce import Salesforce
from io import StringIO
from pandas import DataFrame

In [ ]:
#sf = Salesforce(username='',password='', security_token='')

sf_instance = 'https://ua-trellis.lightning.force.com/' #Your Salesforce Instance URL
reportId = '00O6R000008QxpyUAC' # add report id
export = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
sfUrl = sf_instance + reportId + export
response = requests.get(sfUrl, headers=sf.headers, cookies={'sid': sf.session_id})
download_report = response.content.decode('utf-8')
df1 = pd.read_csv(StringIO(download_report))

In [2]:
os.getcwd()

'/Users/jungmeepark/Documents/Trellis/Network_Analysis_Tableau'

directory = '/Users/jungmeepark/Documents/Trellis/Network_Analysis_Tableau/data/salesforce/'

for file_name in glob.glob(directory+'*.csv'):
    x = np.genfromtxt(file_name,delimiter=',')[:,2]
    # do your calculations


# Define the directory path
as /Users/jungmeepark/Documents/Trellis/Network_Analysis_Tableau/data/salesforce
directory = '/Users/jungmeepark/Documents/Trellis/Network_Analysis_Tableau/data/salesforce/'

# Get a list of all files in the directory
files = os.listdir(directory)

# Filter out only the CSV files
csv_files = [file for file in files if file.endswith('.csv')]

# Loop through each CSV file and read it
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    # Now you can work with the DataFrame 'df' for each CSV file
    print("Loaded:", file)



In [3]:
# Read in Source File - NB this must match the schema requirements
# read in data
# /Users/jungmeepark/Documents/Trellis/Network_Analysis_Tableau/data/salesforce
cases_report_SF = pd.read_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/salesforce/POC_Network_Analysis_Cases.csv')
initial_report_SF = pd.read_csv("~/Documents/Trellis/Network_Analysis_Tableau/data/salesforce/practice_appointments_POC.csv")
#CodeType = 'latin1' # https://docs.python.org/3/library/codecs.html#standard-encodings
print(initial_report_SF.tail())


                    Appointment Name  Case Number Case Detail Name  \
135008           4/26/2024 - Amy You    2704874.0       CD-1536328   
135009       4/30/2024 - Emma Zimmer    2709749.0       CD-1537726   
135010     4/29/2024 - Gabriel Sulit    2706052.0       CD-1536539   
135011        4/26/2024 - Nadia Rana    2710257.0       CD-1537821   
135012  4/25/2024 - Dalton Nonweiler    2702640.0       CD-1535806   

        Owner: Owner ID  Contact EmplID          Case ID Appointment Date  \
135008  0053n0000051PtI      23966508.0  5006R00001zXqX7          4/26/24   
135009  0053n0000051PtI      23966807.0  5006R00001wg9jB          4/30/24   
135010  0053n000007s3zc      23966924.0  5006R00001wfq8c          4/29/24   
135011  0053n000007s3zc      23966958.0  5006R00001wgATn          4/26/24   
135012  0053n0000076YOh      23967405.0  5006R00001zPEXF          4/25/24   

          Advisor Name  Plan/Subplan: CPP Info Name             Advisor Email  
135008   Sofia Meckler              

In [ ]:
print(cases_report_SF.tail())

In [ ]:
print(initial_report_SF.head())

In [4]:
# Filter out rows where case record type and department is NA or empty
cases_report_SF = cases_report_SF.dropna(subset=['Case Record Type', 'Department'])
cases_report_SF.shape

(187600, 10)

In [5]:
# Filter out rows where `Case ID` or `Contact EmplID` is NA or empty
initial_report_SF = initial_report_SF.dropna(subset=['Case Number', 'Case ID', 'Contact EmplID'])
initial_report_SF.shape

(61179, 10)

In [6]:
# trim down the cases to make the merge possible
initial_report_SF = initial_report_SF[
    (~initial_report_SF['Case ID'].isna()) | (initial_report_SF['Case ID'] != "")
]

initial_report_SF = initial_report_SF[
    (~initial_report_SF['Contact EmplID'].isna()) | (initial_report_SF['Contact EmplID'] != "")
]

cases_report_SF = cases_report_SF[
    (~cases_report_SF['Case Record Type'].isna()) | (cases_report_SF['Case Record Type'] != "")
]

cases_report_SF = cases_report_SF[
    (~cases_report_SF['Department'].isna()) | (cases_report_SF['Department'] != "")
]


In [7]:
# Merge the DataFrames
filtered_df = pd.merge(cases_report_SF, initial_report_SF, how='left', left_on='Emplid', right_on='Contact EmplID')


In [8]:
filtered_df.shape

(288882, 20)

In [ ]:
# Filter out rows where `Case ID` or `Contact EmplID` is NA or empty
#filtered_df = merged_df.dropna(subset=['Case Number', 'Case ID', 'Contact EmplID'])
#initial_report_SF = initial_report_SF[(~initial_report_SF['Case ID'].isna()) | (initial_report_SF['Case ID'] != "")]
#initial_report_SF = initial_report_SF[(~initial_report_SF['Contact EmplID'].isna()) | (initial_report_SF['Contact EmplID'] != "")]
#print(filtered_df.head())

In [9]:
# Convert 'Appointment Date' column to datetime
filtered_df['AppointmentDate'] = pd.to_datetime(filtered_df['Appointment Date'])

# Calculate today's date
today = pd.to_datetime('today').normalize()
filtered_df['today'] = today

# Calculate days since 'AppointmentDate'
filtered_df['days_since'] = (filtered_df['today'] - filtered_df['AppointmentDate']).dt.days

# filter dates to 2024
#filtered_df[(filtered_df['date']>datetime.date(2024,1,1)) & (filtered_df['date']<datetime.date(2024,5,5))]
filtered_df = filtered_df[(filtered_df['AppointmentDate'] > "2024-01-01") & (filtered_df['AppointmentDate'] < "2024-07-01")]
# Display the updated DataFrame
print(filtered_df.head())

                        Subject Date/Time Opened  Age (Hours)  Open  Closed  \
4           Campus Pantry Visit    8/30/23 11:25            0     0       1   
9   [CATMAIL] Re: Selecting GRO    10/9/23 21:58           11     0       1   
10  [CATMAIL] Re: Selecting GRO    10/9/23 21:58           11     0       1   
14         Petition Spring 2024   10/19/23 17:52           87     0       1   
15         Petition Spring 2024   10/19/23 17:52           87     0       1   

                         Department Case Record Type  \
4   Basic Needs Center and Services          Service   
9                         Registrar          Service   
10                        Registrar          Service   
14                        Registrar          Service   
15                        Registrar          Service   

              Contact: Email      Emplid   Case Last Modified By  ...  \
4       alexa049@arizona.edu  23717556.0  Public Site Guest User  ...   
9     jae31lopez@arizona.edu  23652401.0  

In [10]:
# create new emplID's
# Extract unique IDs
unique_ids = filtered_df['Contact EmplID'].unique()

# Generate new IDs (random IDs)
np.random.seed(53013)  # for reproducibility
new_ids = np.random.randint(1000, 9999, len(unique_ids))

# Create a dictionary to map old IDs to new IDs
id_mapping = dict(zip(unique_ids, new_ids))

id_mapping
#filtered_df_newID=filtered_df.append(id_mapping,ignore_index=True)


{23717556.0: 4852,
 23652401.0: 6239,
 23778079.0: 7151,
 23555590.0: 7072,
 23690965.0: 7176,
 23851240.0: 3311,
 23807978.0: 3872,
 23835530.0: 6729,
 23631758.0: 5275,
 23863061.0: 5000,
 23823611.0: 8673,
 23575604.0: 6469,
 23870839.0: 9728,
 23857014.0: 8573,
 23740244.0: 8143,
 23816490.0: 7141,
 23781972.0: 2759,
 23785309.0: 3115,
 23601175.0: 8737,
 23793830.0: 8372,
 23748428.0: 2847,
 23799530.0: 1218,
 23701056.0: 9547,
 23872845.0: 7306,
 23669087.0: 5778,
 23731017.0: 7210,
 23620255.0: 6122,
 23687266.0: 6612,
 23700668.0: 6822,
 23874577.0: 1997,
 23724422.0: 5922,
 23652339.0: 3198,
 23600229.0: 6411,
 23573982.0: 7200,
 23741013.0: 2188,
 23602975.0: 5157,
 23870810.0: 5230,
 23851571.0: 8688,
 23838176.0: 6470,
 23787877.0: 8624,
 23848399.0: 7798,
 23793911.0: 3182,
 23777123.0: 3280,
 23732077.0: 3557,
 23846494.0: 3853,
 23673580.0: 2775,
 23734420.0: 8606,
 23461436.0: 1239,
 23766329.0: 8509,
 23623619.0: 7826,
 23838268.0: 9616,
 23810161.0: 6143,
 23722842.0:

In [11]:
#filtered_df['Contact EmplID'].map(id_mapping)  
filtered_df['New_ID'] = filtered_df['Contact EmplID'].map(id_mapping)

In [12]:
# Get variable names (column names) of the DataFrame
variable_names = filtered_df.columns

print(variable_names)

Index(['Subject', 'Date/Time Opened', 'Age (Hours)', 'Open', 'Closed',
       'Department', 'Case Record Type', 'Contact: Email', 'Emplid',
       'Case Last Modified By', 'Appointment Name', 'Case Number',
       'Case Detail Name', 'Owner: Owner ID', 'Contact EmplID', 'Case ID',
       'Appointment Date', 'Advisor Name', 'Plan/Subplan: CPP Info Name',
       'Advisor Email', 'AppointmentDate', 'today', 'days_since', 'New_ID'],
      dtype='object')


In [13]:
# create a new column to extract the names
# Extract just the names
filtered_df['Student Name'] = filtered_df['Appointment Name'].str.split(' - ', expand=True)[1]

# Display the updated DataFrame
print(filtered_df.head())

                        Subject Date/Time Opened  Age (Hours)  Open  Closed  \
4           Campus Pantry Visit    8/30/23 11:25            0     0       1   
9   [CATMAIL] Re: Selecting GRO    10/9/23 21:58           11     0       1   
10  [CATMAIL] Re: Selecting GRO    10/9/23 21:58           11     0       1   
14         Petition Spring 2024   10/19/23 17:52           87     0       1   
15         Petition Spring 2024   10/19/23 17:52           87     0       1   

                         Department Case Record Type  \
4   Basic Needs Center and Services          Service   
9                         Registrar          Service   
10                        Registrar          Service   
14                        Registrar          Service   
15                        Registrar          Service   

              Contact: Email      Emplid   Case Last Modified By  ...  \
4       alexa049@arizona.edu  23717556.0  Public Site Guest User  ...   
9     jae31lopez@arizona.edu  23652401.0  

In [14]:
# Get variable names (column names) of the DataFrame
variable_names = filtered_df.columns

print(variable_names)

Index(['Subject', 'Date/Time Opened', 'Age (Hours)', 'Open', 'Closed',
       'Department', 'Case Record Type', 'Contact: Email', 'Emplid',
       'Case Last Modified By', 'Appointment Name', 'Case Number',
       'Case Detail Name', 'Owner: Owner ID', 'Contact EmplID', 'Case ID',
       'Appointment Date', 'Advisor Name', 'Plan/Subplan: CPP Info Name',
       'Advisor Email', 'AppointmentDate', 'today', 'days_since', 'New_ID',
       'Student Name'],
      dtype='object')


In [15]:
# add an index column

filtered_df['index'] = range(1, len(filtered_df) + 1)
#filtered_df['index'] = filtered_df.index

# Display the updated DataFrame
#print(filtered_df.head())

In [16]:
# create a source column
filtered_df['Source'] = filtered_df.groupby(['Advisor Name']).ngroup()

# Display the updated DataFrame
print(filtered_df.head())

                        Subject Date/Time Opened  Age (Hours)  Open  Closed  \
4           Campus Pantry Visit    8/30/23 11:25            0     0       1   
9   [CATMAIL] Re: Selecting GRO    10/9/23 21:58           11     0       1   
10  [CATMAIL] Re: Selecting GRO    10/9/23 21:58           11     0       1   
14         Petition Spring 2024   10/19/23 17:52           87     0       1   
15         Petition Spring 2024   10/19/23 17:52           87     0       1   

                         Department Case Record Type  \
4   Basic Needs Center and Services          Service   
9                         Registrar          Service   
10                        Registrar          Service   
14                        Registrar          Service   
15                        Registrar          Service   

              Contact: Email      Emplid   Case Last Modified By  ...  \
4       alexa049@arizona.edu  23717556.0  Public Site Guest User  ...   
9     jae31lopez@arizona.edu  23652401.0  

In [17]:
filtered_df = filtered_df.rename(columns={'Owner: Owner ID': 'owner_id', 'New_ID': 'Target'})

In [18]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65251 entries, 4 to 288836
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Subject                      65157 non-null  object        
 1   Date/Time Opened             65251 non-null  object        
 2   Age (Hours)                  65251 non-null  int64         
 3   Open                         65251 non-null  int64         
 4   Closed                       65251 non-null  int64         
 5   Department                   65251 non-null  object        
 6   Case Record Type             65251 non-null  object        
 7   Contact: Email               65251 non-null  object        
 8   Emplid                       65251 non-null  float64       
 9   Case Last Modified By        65251 non-null  object        
 10  Appointment Name             65251 non-null  object        
 11  Case Number                  65251 non-n

In [19]:
filtered_df['Source'] = filtered_df['Source'].map(str)
filtered_df['Target'] = filtered_df['Target'].map(str)

In [20]:
# Add directions to d3
filtered_df['Direction'] = filtered_df['Source'] + ' -> ' + filtered_df['Target']

In [21]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65251 entries, 4 to 288836
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Subject                      65157 non-null  object        
 1   Date/Time Opened             65251 non-null  object        
 2   Age (Hours)                  65251 non-null  int64         
 3   Open                         65251 non-null  int64         
 4   Closed                       65251 non-null  int64         
 5   Department                   65251 non-null  object        
 6   Case Record Type             65251 non-null  object        
 7   Contact: Email               65251 non-null  object        
 8   Emplid                       65251 non-null  float64       
 9   Case Last Modified By        65251 non-null  object        
 10  Appointment Name             65251 non-null  object        
 11  Case Number                  65251 non-n

In [22]:
# Group by 'NAME' and check if any row within the group has a non-null 'DEPARTMENT'
grouped = filtered_df.groupby('Advisor Name')['Department'].transform('first')

In [23]:
# Impute the 'DEPARTMENT' value within each group
filtered_df['Department'] = filtered_df['Department'].fillna(grouped)

In [24]:
# FOLLOWING KNIPPENBERG

# Save the DataFrame to a CSV file

filtered_df.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/py-filtered_merged_df.csv', index=False)  # Set index=False to exclude the index from the CSV file


In [ ]:
#reduced_df = filtered_df.sample(n=300)

In [25]:
arr_SrcTgt = np.array(filtered_df[['Source','Target']])

In [26]:
# Create Network Graph Coordinates...
Q = nx.MultiDiGraph()
Q.add_edges_from(arr_SrcTgt)
dict_Coords = nx.spring_layout(Q) 


In [27]:
# Create Graph Coordinates File...
df_Raw_Coords = DataFrame(dict_Coords)
df_Raw_Coords = df_Raw_Coords.T
df_Raw_Coords.columns = ['X','Y']
df_Raw_Coords.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/CoordsFile_merged.csv', 
                     index_label='NodeName')
#print(df_Raw_Coords)

In [28]:
# Create Bridge File... 
# Tableau Code: IF [Src-Tgt]/2 = ROUND([Src-Tgt]/2) THEN 'Source' ELSE 'Target' END
arr_SrcTgt2 = arr_SrcTgt.reshape(1,(len(arr_SrcTgt)*2)) 
arr_SrcTgt2 = arr_SrcTgt2.reshape(-1) 
df_SrcTgt = DataFrame(arr_SrcTgt2,columns=['NodeName']) 
arr_Index = []
for i in range(1,(len(arr_SrcTgt)+1)):
    arr_Index.append(i)
    arr_Index.append(i)
df_SrcTgt['c_Index'] = arr_Index 

df_SrcTgt.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/BridgeFile_merged.csv',
                 index_label='Src-Tgt')
#print(df_ScrTgt.head())

In [ ]:
nx.draw(dict_Coords)
plt.show()

In [ ]:
#nx.draw(Q, pos=nx.spring_layout(Q));
nx.draw(Q, pos=nx.spring_layout(Q), with_labels=True, node_size=100, node_color="skyblue", edge_color="gray")
plt.show()

In [ ]:
# betweenness centrality
plt.hist(nx.centrality.closeness_centrality(Q).values());

In [ ]:
# nx.diameter(Q)

In [ ]:
# nx.cluster.average_clustering(Q)

In [ ]:
# degree distribution (a histogram of how many edges each node has)

plt.hist([v for k,v in nx.degree(Q)]);

In [ ]:

degrees = [v for k, v in nx.degree(Q)]

# Calculate the range of your data
min_degree = min(degrees)
max_degree = max(degrees)

# Calculate the number of bins based on increments of 10
num_bins = int((max_degree - min_degree) / 10) + 1

# Plot the histogram
plt.hist(degrees, bins=num_bins)
plt.xlim(0, 200)
plt.show()

In [ ]:
print('Run Completed Successfully')

In [ ]:
degrees = [v for k, v in nx.degree(Q)]

# Calculate the range of your data
min_degree = min(degrees)
max_degree = max(degrees)  # Truncate the maximum degree to 600

# Calculate the number of bins based on increments of 5
num_bins = int((max_degree - min_degree) / 5) + 1

# Plot the histogram
plt.hist(degrees, bins=num_bins)
plt.xlim(0, 200)  # Set the limit of x-axis from 0 to 600
plt.show()